In [13]:
from sklearn.datasets import load_iris, load_digits, load_wine
from sklearn.model_selection import train_test_split
import numpy as np
import cvxpy as cp
import os
import matplotlib.pyplot as plt

# Load dataset and initialization

In [14]:
# load dataset
data=load_iris()

# Store features matrix in X
X= data.data
#Store target vector in 
y= data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
n_train=X_train.shape[0]
n_test=X_test.shape[0]
d=X_train.shape[1]
num_class = np.unique(y).size
beta=1e-4

In [15]:
# one hot encoding
y_one_hot = np.zeros((n_train, num_class))
y_one_hot[np.arange(n_train),y_train] = 1
y_train=np.copy(y_one_hot)
y_one_hot = np.zeros((n_test, num_class))
y_one_hot[np.arange(n_test),y_test] = 1
y_test=np.copy(y_one_hot)

In [16]:
## Finite approximation of all possible sign patterns
m1=1024
u_vectors = np.random.normal(0, 1, (d,m1))
dmat = (np.matmul(X_train, u_vectors) >= 0)

# Optimal CVX

In [17]:
Uopt1_list = []
Uopt2_list = []
yopt1_list = []
yopt2_list = []
mse=cp.Parameter((1,1))
norm=0
constraints=[]
for i in range(num_class):
    Uopt1_list.append(cp.Variable((d,m1)))
    Uopt2_list.append(cp.Variable((d,m1)))
    yopt1_list.append(cp.sum(cp.multiply(dmat,(X_train@Uopt1_list[i])),axis=1)) 
    yopt2_list.append(cp.sum(cp.multiply(dmat,(X_train@Uopt2_list[i])),axis=1))
    norm+=cp.mixed_norm(Uopt1_list[i].T,2,1)+cp.mixed_norm(Uopt1_list[i].T,2,1)   
constraints+=[cp.multiply((2*dmat-np.ones((n_train,m1))),(X_train@Uopt1_list[i]))>=0]
constraints+=[cp.multiply((2*dmat-np.ones((n_train,m1))),(X_train@Uopt2_list[i]))>=0]

mse=cp.sum(cp.power(cp.norm(cp.vstack(yopt1_list).T-cp.vstack(yopt2_list).T-y_train, 2, axis=1), 2))/2

In [18]:
## Below we use MSE as a performance metric for classification
cost=mse+beta*norm

prob=cp.Problem(cp.Minimize(cost),constraints)
prob.solve(solver="SCS",verbose=True)
cvx_opt=prob.value

print("Convex program objective value (eq (8)): ",cvx_opt)

                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Jun 11 03:08:55 PM: Your problem has 24576 variables, 2 constraints, and 0 parameters.
(CVXPY) Jun 11 03:08:55 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 11 03:08:55 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 11 03:08:55 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 11 03:08:55 PM: Compiling problem (target solver=SCS).
(CVXPY) Jun 11 03:08:55 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

# Calculate test set accuracy

In [19]:
dmat_test = (np.matmul(X_test, u_vectors) >=0)
n_test = X_test.shape[1]

In [20]:
ypred_list = []
yopt1_test=cp.Parameter((n_test,1))
yopt2_test=cp.Parameter((n_test,1))
for i in range(num_class):
    yopt1_test=cp.sum(cp.multiply(dmat_test,(X_test@Uopt1_list[i])),axis=1)
    yopt2_test=cp.sum(cp.multiply(dmat_test,(X_test@Uopt2_list[i])),axis=1)
    ypred_list.append(yopt1_test-yopt2_test)
y_prob = cp.vstack(ypred_list).T.value
y_pred = np.argmax(y_prob, axis=1)
y_test = np.argmax(y_test, axis=1)

In [21]:
from sklearn.metrics import accuracy_score
accu=accuracy_score(y_test, y_pred)*100
print("accuracy:%.2f"%accu + "%")

accuracy:96.67%
